In [ ]:
# Parallel Genetic Algorithm Optimization
# =========================================

from genetic_optimizer import GeneticOptimizer, ParallelEvaluator, get_available_gpus

# Check available GPUs
n_gpus = get_available_gpus()
print(f"Detected {n_gpus} GPU(s)")

# Define evaluation function that will be called by parallel workers
def evaluate_individual(dot_positions, sim_name):
    """
    Evaluate a single individual (set of dot positions).
    This function runs the simulation and calculates fitness.
    
    Parameters
    ----------
    dot_positions : list
        List of (x, y) tuples for dot positions
    sim_name : str
        Name for this simulation
    
    Returns
    -------
    result : dict
        Dictionary with 'fitness' and other metrics
    """
    # Generate MuMax3 script
    script = f"""
n := {len(dot_positions)}

//  Set up universe & geometry ------------
cell_size_x := 20e-9
cell_size_y := 20e-9
cell_size_z := 10e-9
device_size_x := 5e-6
device_size_y := 1e-6
device_size_z := 100e-9
SetGridsize(250, 50, 20)
SetCellsize(cell_size_x, cell_size_y, cell_size_z)
main_arena := cuboid(3e-6, 1e-6, 100e-9).transl(-1e-6, 0, -50e-9)
output_corridor_top := (cuboid(2e-6, 0.3e-6, 100e-9).transl(1.5e-6, 0.35e-6, 0)).transl(0, 0, -50e-9)
output_corridor_bottom := (cuboid(2e-6, 0.3e-6, 100e-9).transl(1.5e-6, -0.35e-6, 0)).transl(0, 0, -50e-9)
device := (main_arena.add(output_corridor_top)).add(output_corridor_bottom)

// Define material parameters -------------
edge_alpha := 0.5
device_alpha := 2e-4
soft_alpha_edges := 1
Msat = 1.4e5
Aex = 3.5e-12

// Exponentially decrease alpha inside the device ---
if soft_alpha_edges==1{{
    alpha = edge_alpha
    delta := 250e-9
    k := -1e7
    a := (device_alpha - edge_alpha)/(exp(k*delta) - 1)
    b := edge_alpha - a
    temp_device := device
    
    Lx0 := device_size_x
    Ly0 := device_size_y
    
    for i:=1; i<254; i+=1{{ 
        x := delta*i/253
        new_alpha := a*exp(k*x)+b
        
        scale_x := (Lx0 - 2*i*(delta/253))/(Lx0 - 2*(i-1)*(delta/253))
        scale_y := (Ly0 - 2*i*(delta/253))/(Ly0 - 2*(i-1)*(delta/253))
        
        temp_device = temp_device.scale(scale_x, scale_y, 1.0)
        defRegion(i,temp_device)
        alpha.setRegion(i,new_alpha)
    }}
}} else {{
    alpha = device_alpha 
}}

// Add magnetic dots above device (region 254)
dots := cylinder(0, 0)
if n != 0 {{
{chr(10).join([f"    dots = dots.add((cylinder(100e-9,100e-9).transl({x:.15e}, {y:.15e}, 0)).transl(0, 0, 50e-9))" for x, y in dot_positions])}
    defregion(254, dots)
    Msat.setRegion(254, 1.145e6)
    Aex.setRegion(254, 7.5e-12)
    alpha.setRegion(254, 0.2)
}} 

// Set the geometry -----------------------
totalgeom := device.add(dots)
setgeom(totalgeom)

// Simulation parameters ------------------
T = 50e-9
f1 := 2.6e9
f2 := 2.8e9
sample_dt := 50e-12
m = uniform(0.02, 0.02, 1)
B_ext = vector(0, 0, 0.2)
autosave(m,sample_dt)

// Input stripline (region 255) -----------
input_stripline := cuboid(300e-9, 0.8e-6, 100e-9).transl(-2.5e-6 + 150e-9, 0, -50e-9)
defregion(255,input_stripline)
B_ext.setregion(255, vector(0, (0.1e-3)*sin(2*pi*f1*t) + (0.1e-3)*sin(2*pi*f2*t), 0.2))

// Run ------------------------------------
run(T)
"""
    
    # Run simulation
    table, fields = run_mumax3(script, name=sim_name, verbose=False)
    
    # Build magnetization array and perform FFT
    M, timestamps = build_magnetization_array(fields, pattern='m')
    maps, fpos = mag_tfft_select(M, component='z', dt=50e-12, fsel=[2.6e9, 2.8e9], 
                                 dimorder='zyx', detrend=True, window='hann', stat='amp')
    
    # Measure outputs
    measurement_size = 300e-9
    output_top_center = (1.5e-6, 0.35e-6)
    output_bottom_center = (1.5e-6, -0.35e-6)
    device_size_x = 5e-6
    device_size_y = 1e-6
    
    results = {}
    for i, (fmap, f) in enumerate(zip(maps, fpos)):
        freq_label = f"{f/1e9:.2f} GHz"
        
        amp_top, _ = measure_region_amplitude(
            fmap, output_top_center[0], output_top_center[1], 
            measurement_size, device_size_x, device_size_y
        )
        
        amp_bottom, _ = measure_region_amplitude(
            fmap, output_bottom_center[0], output_bottom_center[1],
            measurement_size, device_size_x, device_size_y
        )
        
        results[freq_label] = {'top': amp_top, 'bottom': amp_bottom}
    
    # Calculate fitness (overall selectivity score)
    f1_label = f"{fpos[0]/1e9:.2f} GHz"
    f2_label = f"{fpos[1]/1e9:.2f} GHz"
    
    selectivity_top = results[f1_label]['top'] / (results[f2_label]['top'] + 1e-10)
    selectivity_bottom = results[f2_label]['bottom'] / (results[f1_label]['bottom'] + 1e-10)
    fitness = selectivity_top * selectivity_bottom
    
    return {
        'fitness': fitness,
        'selectivity_top': selectivity_top,
        'selectivity_bottom': selectivity_bottom,
        'results': results
    }


# Initialize optimizer
optimizer = GeneticOptimizer(
    population_size=10,      # 10 individuals per generation
    n_dots=8,                # 8 FeRh dots to optimize
    x_range=(-2.4e-6, -0.1e-6),
    y_range=(-0.4e-6, 0.4e-6),
    mutation_rate=0.2,
    mutation_std=100e-9,
    crossover_rate=0.7,
    elite_fraction=0.2
)

# Generate initial population
optimizer.initialize_population(seed=42)

# Create parallel evaluator (will use all available GPUs)
parallel_eval = ParallelEvaluator(evaluate_individual, n_workers=n_gpus)

# Number of generations to run
n_generations = 5

print(f"\n{'='*60}")
print(f"Starting PARALLEL optimization for {n_generations} generations")
print(f"Population size: {optimizer.population_size}")
print(f"Workers (GPUs): {parallel_eval.n_workers}")
print(f"{'='*60}\n")

# Main optimization loop
for gen in range(n_generations):
    print(f"\n{'='*60}")
    print(f"GENERATION {optimizer.generation}")
    print(f"{'='*60}\n")
    
    # Get current population
    population = optimizer.get_current_population()
    
    # Evaluate entire population IN PARALLEL
    fitness_scores = parallel_eval.evaluate_population(population, optimizer.generation)
    
    # Set fitness scores for this generation
    optimizer.set_fitness_scores(fitness_scores)
    
    # Save checkpoint
    optimizer.save_checkpoint(f"optimizer_gen{optimizer.generation}.json")
    
    # Evolve to next generation (unless this is the last one)
    if gen < n_generations - 1:
        optimizer.evolve()

print(f"\n{'='*60}")
print("OPTIMIZATION COMPLETE")
print(f"{'='*60}\n")

# Get best individual
best_dots, best_fitness = optimizer.get_best_individual()
print(f"Best fitness achieved: {best_fitness:.4f}")
print(f"\nBest dot positions:")
for i, (x, y) in enumerate(best_dots):
    print(f"  Dot {i+1}: x={x*1e6:.3f} µm, y={y*1e6:.3f} µm")

In [ ]:
# Visualize optimization progress
# ================================

import matplotlib.pyplot as plt

# Plot fitness evolution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Fitness over generations
ax = axes[0]
generations = optimizer.history['generations']
best_fitness_hist = optimizer.history['best_fitness']
mean_fitness_hist = optimizer.history['mean_fitness']

ax.plot(generations, best_fitness_hist, 'o-', color='green', linewidth=2, 
        markersize=8, label='Best fitness')
ax.plot(generations, mean_fitness_hist, 's-', color='blue', linewidth=2, 
        markersize=6, label='Mean fitness')
ax.set_xlabel('Generation', fontsize=12)
ax.set_ylabel('Fitness (selectivity score)', fontsize=12)
ax.set_title('Optimization Progress (Parallel)', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

# Plot 2: Best dot configuration
ax = axes[1]

# Plot device outline
device_width = 5e-6
device_height = 1e-6
ax.add_patch(plt.Rectangle((-device_width/2*1e6, -device_height/2*1e6), 
                           3e-6*1e6, device_height*1e6, 
                           fill=False, edgecolor='gray', linewidth=2, linestyle='--'))
ax.add_patch(plt.Rectangle((0.5*1e6, 0.2*1e6), 2e-6*1e6, 0.3e-6*1e6,
                           fill=False, edgecolor='gray', linewidth=2, linestyle='--', label='Output corridors'))
ax.add_patch(plt.Rectangle((0.5*1e6, -0.5*1e6), 2e-6*1e6, 0.3e-6*1e6,
                           fill=False, edgecolor='gray', linewidth=2, linestyle='--'))

# Plot best dot positions
for i, (x, y) in enumerate(best_dots):
    circle = plt.Circle((x*1e6, y*1e6), 0.05, color='red', alpha=0.7)
    ax.add_patch(circle)
    ax.text(x*1e6, y*1e6, str(i+1), ha='center', va='center', 
            color='white', fontsize=8, fontweight='bold')

# Plot input stripline
ax.add_patch(plt.Rectangle((-2.5e-6*1e6 + 0.15*1e6, -0.4*1e6), 
                           0.3*1e6, 0.8*1e6, 
                           fill=True, facecolor='cyan', alpha=0.3, 
                           edgecolor='cyan', linewidth=2, label='Input stripline'))

ax.set_xlim(-2.6, 2.6)
ax.set_ylim(-0.55, 0.55)
ax.set_aspect('equal')
ax.set_xlabel('x (µm)', fontsize=12)
ax.set_ylabel('y (µm)', fontsize=12)
ax.set_title(f'Best Configuration (Fitness: {best_fitness:.4f})', 
             fontsize=14, fontweight='bold')
ax.legend(fontsize=10, loc='upper right')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('optimization_results_parallel.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nFitness improvement: {best_fitness/optimizer.history['best_fitness'][0]:.2f}x")
print(f"Initial best: {optimizer.history['best_fitness'][0]:.4f}")
print(f"Final best:   {best_fitness:.4f}")